## LLM Flow

#### Imports 

In [1]:
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts.chat import AIMessagePromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chat_models import ChatVertexAI
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from google.cloud import aiplatform
from google.cloud import bigquery
import pandas as pd
import langchain
import datetime
import logging
import google

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

#### Essentials 

In [5]:
PROJECT = 'arun-genai-bb'
LOCATION = 'us-central1'
MODEL_NAME = 'chat-bison'

In [6]:
bq = bigquery.Client()

In [7]:
llm = ChatVertexAI(project=PROJECT, 
                   location=LOCATION, 
                   model_name=MODEL_NAME,
                   temperature=0.0, 
                   max_output_tokens=256)

### Step 1: Identify the `intent` of the user's query

##### Load example prompt and completion pairs needed for intent detection

In [8]:
messages = []

In [9]:
examples = pd.read_csv('./../DATA/Example-1/hotel_reservation_intent_prompts.csv')
examples.head()

,Prompt,Intent
0,I'm looking to book a single room in Orlando f...,MAKE_RESERVATION
1,Can I reserve a suite in New York from Decembe...,MAKE_RESERVATION
2,I'd like to change my reservation from a doubl...,MODIFY_RESERVATION
3,Is it possible to cancel my booking for the we...,CANCEL_RESERVATION
4,Do you have any available rooms with a sea vie...,CHECK_AVAILABILITY


In [10]:
template = "You are a helpful assistant capable of detecting the intent behind a user's query."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
messages.append(system_message_prompt)

In [11]:
for _, row in examples.iterrows():
    prompt, completion = row
    human_message = HumanMessagePromptTemplate.from_template(prompt)
    messages.append(human_message)
    ai_message = AIMessagePromptTemplate.from_template(completion)
    messages.append(ai_message)

In [12]:
human_template = "{user_query}"
human_message = HumanMessagePromptTemplate.from_template(human_template)
messages.append(human_message)

In [13]:
chat_prompt = ChatPromptTemplate.from_messages(messages)

In [14]:
user_query = "I would like to make a reservation in Miami for four nights from September 23rd to the 28th"
request = chat_prompt.format_prompt(user_query=user_query).to_messages()

In [15]:
%%time 

response = llm(request)

CPU times: user 26.2 ms, sys: 4.33 ms, total: 30.5 ms
Wall time: 2.55 s


In [16]:
intent = response.content.strip()
logger.info(intent)

MAKE_RESERVATION


### Step 2: Extract the entities from the user query

Load example prompt and completion pairs needed for entity recognition

In [17]:
messages = []

In [18]:
examples = pd.read_csv('./../DATA/Example-1/hotel_reservation_ner_prompts.csv')
examples.head()

,Prompt,Extracted Entities
0,I need to book a room at Hilton in London from...,Hotel Brand:Hilton|Location:London|Start Date:...
1,Please find availability at Marriott in New Yo...,Hotel Brand:Marriott|Location:New York|Start D...
2,I'm planning to stay at the Hyatt in Paris fro...,Hotel Brand:Hyatt|Location:Paris|Start Date:De...
3,Can you check rates for Sheraton in Rome for t...,Hotel Brand:Sheraton|Location:Rome|Start Date:...
4,I am looking for rooms at the Ritz-Carlton in ...,Hotel Brand:Ritz-Carlton|Location:Tokyo|Start ...


In [19]:
template = "You are a helpful assistant capable of performing named entity recognition."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
messages.append(system_message_prompt)

In [20]:
for _, row in examples.iterrows():
    prompt, completion = row
    human_message = HumanMessagePromptTemplate.from_template(prompt)
    messages.append(human_message)
    ai_message = AIMessagePromptTemplate.from_template(completion)
    messages.append(ai_message)

In [21]:
human_template = "{user_query} Standardize the date format to YYYY-MM-DD."
human_message = HumanMessagePromptTemplate.from_template(human_template)
messages.append(human_message)

In [22]:
chat_prompt = ChatPromptTemplate.from_messages(messages)

In [23]:
user_query = "I would like to make a reservation in Miami for four nights from September 23rd to the 28th for 2023."
request = chat_prompt.format_prompt(user_query=user_query).to_messages()

In [24]:
%%time 

response = llm(request)

CPU times: user 5.47 ms, sys: 216 µs, total: 5.68 ms
Wall time: 1.04 s


In [25]:
entities = response.content.strip()
logger.info(entities)

Location:Miami|Start Date:2023-09-23|End Date:2023-09-28


### Step 3: Map Intent to Database Action

In [26]:
messages = []

In [27]:
examples = pd.read_csv('./../DATA/Example-1/intent_to_tables.csv')
examples.head()

,Intent,Tables
0,MAKE_RESERVATION,reservations|rooms|hotels
1,CANCEL_RESERVATION,reservations
2,UPDATE_RESERVATION,reservations|rooms
3,GET_HOTEL_INFO,hotels
4,GET_ROOM_DETAILS,rooms


In [28]:
template = "You are a helpful assistant capable of mapping detected intent to the correct list of BigQuery tables."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
messages.append(system_message_prompt)

In [29]:
for _, row in examples.iterrows():
    prompt, completion = row
    human_message = HumanMessagePromptTemplate.from_template(prompt)
    messages.append(human_message)
    ai_message = AIMessagePromptTemplate.from_template(completion)
    messages.append(ai_message)

In [30]:
human_template = "{user_intent}"
human_message = HumanMessagePromptTemplate.from_template(human_template)
messages.append(human_message)

In [31]:
chat_prompt = ChatPromptTemplate.from_messages(messages)

In [32]:
request = chat_prompt.format_prompt(user_intent=intent).to_messages()

In [33]:
%%time 

response = llm(request)

CPU times: user 4.71 ms, sys: 715 µs, total: 5.42 ms
Wall time: 433 ms


In [34]:
tables = response.content.strip()
logger.info(tables)

reservations|rooms|hotels


### Step 4: SQL Creation

Load table descriptions as additional context to help SQL creation

In [35]:
with open("./../DATA/Example-1/descriptions/hotels.txt", "rb") as f:
    hotels_table_desc = f.read()

In [36]:
with open("./../DATA/Example-1/descriptions/rooms.txt", "rb") as f:
    rooms_table_desc = f.read()

In [37]:
with open("./../DATA/Example-1/descriptions/reservations.txt", "rb") as f:
    reservations_table_desc = f.read()

**Note:** Load descriptions based on mapped tables from the previous step. Do not load all of the descriptions.

In [38]:
messages = []

In [39]:
template = "You are a SQL master expert capable of writing complex SQL query in BigQuery."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
messages.append(system_message_prompt)

In [40]:
human_template = """Given the following inputs:
- INTENT: {intent}
- EXTRACTED_ENTITIES: {entities}
- MAPPED_TABLES: {tables}

Please construct a SQL query using the MAPPED_TABLES. 
The goal is to determine the availability of hotels based on the EXTRACTED_ENTITIES. 
Extract all info regarding the available hotels and rooms including its price.

Below are the schema descriptions for the relevant tables:

**Hotels Table:**
{hotels_table_desc}

**Rooms Table:**
{rooms_table_desc}

**Reservations Table:**
{reservations_table_desc}

Note: Prefix the table names with `hotel_reservations."""

In [41]:
human_message = HumanMessagePromptTemplate.from_template(human_template)
messages.append(human_message)

In [42]:
messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='You are a SQL master expert capable of writing complex SQL query in BigQuery.', template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['entities', 'hotels_table_desc', 'intent', 'reservations_table_desc', 'rooms_table_desc', 'tables'], output_parser=None, partial_variables={}, template='Given the following inputs:\n- INTENT: {intent}\n- EXTRACTED_ENTITIES: {entities}\n- MAPPED_TABLES: {tables}\n\nPlease construct a SQL query using the MAPPED_TABLES. \nThe goal is to determine the availability of hotels based on the EXTRACTED_ENTITIES. \nExtract all info regarding the available hotels and rooms including its price.\n\nBelow are the schema descriptions for the relevant tables:\n\n**Hotels Table:**\n{hotels_table_desc}\n\n**Rooms Table:**\n{rooms_table_desc}\n\n**Reservations Table

In [43]:
chat_prompt = ChatPromptTemplate.from_messages(messages)

In [44]:
request = chat_prompt.format_prompt(intent=intent, 
                                    entities=entities, 
                                    tables=tables, 
                                    hotels_table_desc=hotels_table_desc, 
                                    rooms_table_desc=rooms_table_desc, 
                                    reservations_table_desc=reservations_table_desc).to_messages()

In [45]:
%%time 

response = llm(request)

CPU times: user 6.22 ms, sys: 957 µs, total: 7.18 ms
Wall time: 4.21 s


In [46]:
sql = '\n'.join(response.content.strip().split('\n')[1:-1])
logger.info(sql)

SELECT 
  h.hotel_name,
  r.room_type,
  r.price_per_night,
  r.availability
FROM hotel_reservations.hotels h
JOIN hotel_reservations.rooms r
ON h.hotel_id = r.hotel_id
WHERE h.location = 'Miami'
AND r.availability > 0
AND NOT EXISTS (
  SELECT *
  FROM hotel_reservations.reservations res
  WHERE res.room_id = r.room_id
  AND (
    (res.start_date BETWEEN '2023-09-23' AND '2023-09-28')
    OR (res.end_date BETWEEN '2023-09-23' AND '2023-09-28')
    OR ('2023-09-23' BETWEEN res.start_date AND res.end_date)
    OR ('2023-09-28' BETWEEN res.start_date AND res.end_date)
  )
)


### Step 5: Execute the SQL query in BigQuery

In [47]:
query_job = bq.query(sql)

In [48]:
sql_result = []

for row in query_job:
    sql_result.append(f"Hotel name = {row.hotel_name}")
    sql_result.append(f"Room type = {row.room_type}")
    sql_result.append(f"Price per night in $ = {row.price_per_night}")
    sql_result.append(f"Number of rooms available = {row.availability}")
    sql_result.append("")

sql_result = "\n".join(sql_result).strip()
logger.info(sql_result)

Hotel name = Beachfront Resort
Room type = Deluxe
Price per night in $ = 180.0
Number of rooms available = 10

Hotel name = Beachfront Resort
Room type = Suite
Price per night in $ = 280.0
Number of rooms available = 5


### Step 6: Transform SQL results into a human friendly response

In [49]:
messages = []

In [50]:
template = "You are a travel assistant chatbot that can help people make travel reservations."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
messages.append(system_message_prompt)

In [51]:
human_template = """{user_query}

Given the following list of room availabilities as below:
{sql_result}

Please transform them into a human-readable format."""

In [52]:
human_message = HumanMessagePromptTemplate.from_template(human_template)
messages.append(human_message)

In [53]:
chat_prompt = ChatPromptTemplate.from_messages(messages)

In [54]:
request = chat_prompt.format_prompt(user_query=user_query,
                                    sql_result=sql_result).to_messages()

In [55]:
%%time 

response = llm(request)

CPU times: user 4.47 ms, sys: 685 µs, total: 5.15 ms
Wall time: 1.36 s


In [56]:
output = response.content.strip()
logger.info(output)

Here are the room availabilities for your requested stay at the Beachfront Resort:

**Deluxe Room**

* Price per night: $180.00
* Number of rooms available: 10

**Suite**

* Price per night: $280.00
* Number of rooms available: 5
